In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.nn_data_classifier import load_data, Classifier
from utils.preprocess import preprocess, RNNDataset
from utils.nn_data_classifier import Classifier
writer = SummaryWriter(log_dir='logs')


In [2]:
classi = Classifier(max_drop=-0.05, max_raise=0.05, max_period=240)
historical_data = load_data()

historical_data['minute'] = historical_data.Day.dt.minute
historical_data = historical_data[historical_data.minute == 0]
historical_data.drop('minute', axis=1, inplace=True)

classified_data = classi.classify_data(historical_data)
classified_data.to_csv('./historicals/BTC-min-005-005_with_max_period.csv')
classified_data

Interacao atual: 0/44584
Interacao atual: 445/44584
Interacao atual: 890/44584
Interacao atual: 1335/44584
Interacao atual: 1780/44584
Interacao atual: 2225/44584
Interacao atual: 2670/44584
Interacao atual: 3115/44584
Interacao atual: 3560/44584
Interacao atual: 4005/44584
Interacao atual: 4450/44584
Interacao atual: 4895/44584
Interacao atual: 5340/44584
Interacao atual: 5785/44584
Interacao atual: 6230/44584
Interacao atual: 6675/44584
Interacao atual: 7120/44584
Interacao atual: 7565/44584
Interacao atual: 8010/44584
Interacao atual: 8455/44584
Interacao atual: 8900/44584
Interacao atual: 9345/44584
Interacao atual: 9790/44584
Interacao atual: 10235/44584
Interacao atual: 10680/44584
Interacao atual: 11125/44584
Interacao atual: 11570/44584
Interacao atual: 12015/44584
Interacao atual: 12460/44584
Interacao atual: 12905/44584
Interacao atual: 13350/44584
Interacao atual: 13795/44584
Interacao atual: 14240/44584
Interacao atual: 14685/44584
Interacao atual: 15130/44584
Interacao atu

,Day,Price,Volume,Classification
525539,2017-01-01 01:00:00,966.60,113.092200,1.0
525479,2017-01-01 02:00:00,964.35,43.352316,1.0
525419,2017-01-01 03:00:00,963.97,0.000000,1.0
525359,2017-01-01 04:00:00,960.60,10.717626,1.0
525299,2017-01-01 05:00:00,963.46,0.000000,1.0
...,...,...,...,...
2064801,2022-02-28 23:00:00,43053.74,129851.644413,2.0
2064741,2022-03-01 00:00:00,43237.60,13215.276262,2.0
2064681,2022-03-01 01:00:00,43548.62,5412.571754,2.0
2064621,2022-03-01 02:00:00,43296.76,10547.801236,2.0


In [3]:
classified_data_path = './historicals/BTC-min-005-005_with_max_period.csv'
data = pd.read_csv(classified_data_path, parse_dates=['Day']).iloc[:, 1:]
data.sort_values("Day", inplace=True)
data


,Day,Price,Volume,Classification
0,2017-01-01 01:00:00,966.60,113.092200,1.0
1,2017-01-01 02:00:00,964.35,43.352316,1.0
2,2017-01-01 03:00:00,963.97,0.000000,1.0
3,2017-01-01 04:00:00,960.60,10.717626,1.0
4,2017-01-01 05:00:00,963.46,0.000000,1.0
...,...,...,...,...
44579,2022-02-28 23:00:00,43053.74,129851.644413,2.0
44580,2022-03-01 00:00:00,43237.60,13215.276262,2.0
44581,2022-03-01 01:00:00,43548.62,5412.571754,2.0
44582,2022-03-01 02:00:00,43296.76,10547.801236,2.0


In [4]:
SEQ_LEN = 300

In [5]:
processor = preprocess(sequence_length=SEQ_LEN)
training, validation  = processor.preprocess(dataframe=data, validation_size=0.2)
print(len(training))
print(len(validation))

/home/ayrton/Projects/Quant-Trader/utils/preprocess.py:161: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(sequential_data)


35368
8618


In [6]:
cut_size = len(validation) // 2
testing = validation[:cut_size]
validation = validation[cut_size:]

testing = RNNDataset(sequences=testing[0], labels=testing[1])
validation = RNNDataset(sequences=validation[0], labels=validation[1])

print(len(validation))
print(len(testing))

4309
4309


In [7]:
from torch import nn
class RNN_module(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, num_layers):
        super(RNN_module, self).__init__()
        self._num_layers = num_layers
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.lstm = nn.LSTM(input_size = self._input_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)
        self.fc = nn.Linear(in_features=self._hidden_size, out_features= self._output_size)

    def forward(self, input):
        h_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)
        c_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size, dtype=torch.float64)

        lstm_output, (h_n, c_n) = self.lstm(input)

        pred = self.fc(lstm_output[:, -1, :])
        
        return pred

In [8]:
from torch.utils.data import DataLoader

learning_rate = 0.01
dim_size = training._sequence.size(dim=-1)
hidden_size = 60
batch_size = 64
number_of_classes = training._labels.size(dim=-1)

train_dataloader = DataLoader(training, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(validation, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testing, batch_size = batch_size, shuffle = False)

model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 1)

In [9]:
labels = training._labels
classes = torch.unique(labels, dim = 0)
weights = torch.zeros((classes.shape[0],))
total = torch.tensor(len(training))

for idx, _class in enumerate(classes):
    weights = weights + _class * (1 - labels[labels == _class].sum() / total)

weights

tensor([0.6117, 0.4875, 0.9008])

In [10]:
criterion = nn.CrossEntropyLoss(weight= weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
CHECKPOINT_PATH = './models_parameters/LSTM/checkpoints_2/'
BEST_PATH = './models_parameters/LSTM/best_model.pth'

def epoch_training(train_dataloader, epoch, total_epochs):
    running_correct = 0
    running_loss = 0.0
    n_of_steps = len(train_dataloader)

    for current_batch, (sequence, label) in enumerate(train_dataloader):
        #forward: we are calculating the loss given the parameters
        outputs = model(sequence)
        loss = criterion(input=outputs, target = label)

        #backward: lets update the parameters given the current loss
        optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
        loss.backward() #computates the bwrd-prop gradient for each model parameter
        optimizer.step() #updates the model current parameter using the gradients.

        _, predictions = torch.max(outputs, 1)
        _, correct = torch.max(label, 1)

        running_correct += (predictions == correct).sum().item()
        running_loss += loss.item()

        if (current_batch + 1) % 50 == 0:
            print(f"epoch {epoch+1}/{total_epochs}, current step(batch): {current_batch+1}/{n_of_steps}, loss = {loss.item():.4f} ")
            writer.add_scalar('training loss: ', running_loss/50, epoch * n_of_steps + current_batch)
            writer.add_scalar('accuracy: ', running_correct/50, epoch * n_of_steps + current_batch)
            running_loss = 0.0
            running_correct = 0

    writer.add_scalar('Epoch loss: ', loss, epoch + 1)


def epoch_validate(validation_dataloader, epoch, total_epochs):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0
        
        for current_batch, (sequence, label) in enumerate(validation_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            _, predictions = torch.max(outputs, 1)
            
            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

        acc = 100.0 * n_corrects / n_samples

        print(f"epoch {epoch+1}/{total_epochs} accuracy: {acc}")
        writer.add_scalar('Validation Accuracy: ', acc, epoch+1)

    return acc


def train_loop(train_dataloader: DataLoader, validation_dataloader: DataLoader, epochs: int):
    
    n_of_steps = len(train_dataloader) #total number of batches
    running_correct = 0
    running_loss = 0.0
    max_accuracy = 0
    is_best = False

    for epoch in range(epochs):
        epoch_training(train_dataloader, epoch, epochs)

        accuracy = epoch_validate(validation_dataloader, epoch, epochs)

        if accuracy > max_accuracy:
            is_best = True
        
        checkpoint = {
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict()
        }

        if is_best:
            torch.save(checkpoint, BEST_PATH)
        
        torch.save(checkpoint, CHECKPOINT_PATH+f'model_{epoch+1}.pth')

In [18]:
def test_loop(test_dataloader: DataLoader):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(test_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            _, predictions = torch.max(outputs, 1)

            n_samples += outputs.shape[0]
            n_corrects += (predictions == label).sum().item()

            if (current_batch + 1) % 200 == 0:
                print(f"test batch: {current_batch+1}/{len(test_dataloader)}, current accuracy: {100 * n_corrects / n_samples}")

        acc = 100.0 * n_corrects / n_samples
        print(f"final test accuracy: {acc}")

test_loop(test_dataloader)


final test accuracy: 50.82385704339754


In [13]:
epochs = 10
train_loop(train_dataloader=train_dataloader, validation_dataloader = validation_dataloader, epochs=epochs)
test_loop(test_dataloader=test_dataloader) #curioso: quando o dataloader tava em lista (tuple) o modelo tava treinando bem mais rápido...

epoch 1/10, current step(batch): 50/553, loss = 0.5826 
epoch 1/10, current step(batch): 100/553, loss = 0.5713 
epoch 1/10, current step(batch): 150/553, loss = 0.5829 
epoch 1/10, current step(batch): 200/553, loss = 0.4969 
epoch 1/10, current step(batch): 250/553, loss = 0.5466 
epoch 1/10, current step(batch): 300/553, loss = 0.5656 
epoch 1/10, current step(batch): 350/553, loss = 0.5295 
epoch 1/10, current step(batch): 400/553, loss = 0.4761 
epoch 1/10, current step(batch): 450/553, loss = 0.6336 
epoch 1/10, current step(batch): 500/553, loss = 0.5658 
epoch 1/10, current step(batch): 550/553, loss = 0.5839 
epoch 1/10 accuracy: 47.20352750058018
epoch 2/10, current step(batch): 50/553, loss = 0.4974 
epoch 2/10, current step(batch): 100/553, loss = 0.5462 
epoch 2/10, current step(batch): 150/553, loss = 0.6037 
epoch 2/10, current step(batch): 200/553, loss = 0.5667 
epoch 2/10, current step(batch): 250/553, loss = 0.5552 
epoch 2/10, current step(batch): 300/553, loss = 0.

IndexError: tuple index out of range

In [19]:
writer.close()